In [1]:
import pandas as pd
import numpy as np

# ===============================
# 1. 데이터 로드
# ===============================
df = pd.read_csv("../data/driving_log.csv")  # 노트북이 notebooks/ 안에 있을 때 경로

# 컬럼 체크
required_cols = {"time_sec", "speed_kmh", "accel_mps2"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing columns: {missing}")

# ===============================
# 2. 임계값(프로젝트 02와 동일)
# ===============================
HARD_BRAKE_THRESHOLD = -3.0
HARD_ACCEL_THRESHOLD = 3.0

# ===============================
# 3. 이벤트 탐지
# ===============================
hard_brake_df = df[df["accel_mps2"] < HARD_BRAKE_THRESHOLD]
hard_accel_df = df[df["accel_mps2"] > HARD_ACCEL_THRESHOLD]

hard_brake_count = len(hard_brake_df)
hard_accel_count = len(hard_accel_df)
total_hard_events = hard_brake_count + hard_accel_count

# ===============================
# 4. 시간/거리 계산
# ===============================
total_time_sec = df["time_sec"].max() - df["time_sec"].min()
avg_speed_kmh = df["speed_kmh"].mean()
distance_km = avg_speed_kmh * (total_time_sec / 3600) if total_time_sec > 0 else 0

# ===============================
# 5. 지표 계산 (프로젝트 03/04 재사용)
# ===============================
# events per minute (안정성: 60초 미만이면 None)
events_per_min = (total_hard_events / (total_time_sec / 60)) if total_time_sec >= 60 else None

# events per km (거리 0이면 None)
events_per_km = (total_hard_events / distance_km) if distance_km > 0 else None

# 급제동 비율
hard_brake_ratio = (hard_brake_count / total_hard_events) if total_hard_events > 0 else 0

# 속도 변동성(표준편차)
speed_variability = df["speed_kmh"].std()

# ===============================
# 6. 점수 정규화 함수
# ===============================
def normalize(value, max_value):
    """None이면 0으로 처리(데이터 부족 방어)."""
    if value is None or max_value <= 0:
        return 0
    return min(float(value) / float(max_value), 1.0)

# ===============================
# 7. (참고) 공격성 점수(프로젝트 04 방식) 계산
# ===============================
# 가중치는 포트폴리오용 기본값이며, 실제 적용 시 기준에 따라 조정 가능
aggr_score = 0
aggr_score += normalize(events_per_km, 10) * 30
aggr_score += normalize(events_per_min, 5) * 30
aggr_score += hard_brake_ratio * 20
aggr_score += normalize(speed_variability, 20) * 20
aggr_score = round(aggr_score, 1)  # 0~100

# ===============================
# 8. 프로젝트 05: ADAS 개입 필요도 점수 설계
# ===============================
# 핵심: "개입 필요도"는 공격성 + 급제동 비율 + 거리당 이벤트 + 속도 변동성 종합
# - events_per_min은 데이터 부족으로 None이 될 수 있으니, 점수 계산에서 직접 의존도를 낮게 가져감
adas_risk = 0
adas_risk += (aggr_score / 100) * 35                 # 공격성 점수(상위 지표)
adas_risk += normalize(events_per_km, 10) * 25       # 거리당 위험 이벤트
adas_risk += hard_brake_ratio * 25                   # 급제동은 위험도에 더 민감
adas_risk += normalize(speed_variability, 20) * 15   # 속도 변동성

adas_risk_score = round(adas_risk, 1)  # 0~100

# ===============================
# 9. 개입 레벨 분류
# ===============================
def classify_intervention(score):
    if score <= 30:
        return "No Intervention"
    elif score <= 55:
        return "Monitoring Only"
    elif score <= 75:
        return "ADAS Alert Recommended"
    else:
        return "ADAS Active Intervention"

intervention_level = classify_intervention(adas_risk_score)

# ===============================
# 10. 결과 출력
# ===============================
print("ADAS Intervention Score Result")
print("=" * 45)
print(f"Total Time (sec)         : {total_time_sec:.1f}")
print(f"Distance (km)            : {distance_km:.2f}")
print("-" * 45)
print(f"Hard Accel Events        : {hard_accel_count}")
print(f"Hard Brake Events        : {hard_brake_count}")
print(f"Total Hard Events        : {total_hard_events}")
print("-" * 45)
print(f"Events per Minute        : {events_per_min if events_per_min is not None else 'Insufficient Data'}")
print(f"Events per km            : {events_per_km if events_per_km is not None else 'Insufficient Data'}")
print(f"Hard Brake Ratio         : {hard_brake_ratio:.2f}")
print(f"Speed Variability        : {speed_variability:.2f}")
print("-" * 45)
print(f"Aggressiveness Score     : {aggr_score} / 100")
print(f"ADAS Risk Score          : {adas_risk_score} / 100")
print(f"Intervention Level       : {intervention_level}")


FileNotFoundError: [Errno 2] No such file or directory: '../data/driving_log.csv'